In [1]:
# The visualisations for this notebook can be seen here: https://www.kaggle.com/code/andrecrea/netflix-eda/edit/run/94999758 
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Netflix Exploratory Data Analysis****

The Goal of this analysis is to uncover any insights or trends in the catalogue of films and TV shows available on the Netflix website.

First I will import the dataset that to see the parameters I have to work with.

In [2]:
df = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")

In [3]:
df.head()

In [4]:
df.columns

Now we have the data imported, the above line of code shows the columns that we have to work with many of which are self explanatory, but I will highlight a the below:
* Show ID - This tells us what season the show is, if a TV show 
* Type - Whether it's a Movie or TV-Show
* Listed in - The category that the TV-Show  or film falls under e.g. Documentary or Action
* Rating - the audiences that the film or TV-Show is safe for vieiwing for 

Now that the data is imported, the next step is to Process or clean the data for analysis, I will begin first by checking for any missing values.

In [5]:
missing_data = df.isnull()

In [6]:
missing_data.head()

Above I have created a dataframe that tells me for which values are missing, values that are missing are True and those which are not are False. This can be difficult to view so to make it easier to see, let's count the amount of missing data in each column.

In [7]:
for column in missing_data.columns.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("")

Now I can see that there a missing values in the columns; 
* Director
* Cast
* Country
* Date-added
* Rating
* Duration

For some columns I will ignore these missing values, e.g cast and director. But I will fix some of the other columns.
First Ratings:

In [8]:
df['rating'].value_counts().sort_values(ascending = False)

We can see here that some of the ratings have durations instead, this may be an error where the data has been transposed

In [9]:
df['duration'].value_counts().sort_values(ascending=False)


Let's isolate these rows and see what has happened.

In [10]:
print(df.loc[df['rating'] == '66 min'])
print(df.loc[df['rating'] == '84 min'])
print(df.loc[df['rating'] == '74 min'])



There seems to be an error with the titles by the comedian Louis CK, a quick look up on the Netflix website will tell us what the correct rating is for these shows.

In [11]:
#Louis C.K.: Live at the Comedy Store 1 hr 5 min (15 - tvma)
#Louis C.K.: Hilarious 1 hr 23 min (18)
#Louis C.K. 2017 1 hr 14 min (15 - tvma)


Now that we have the correct rating we can now move the values for duration into the correct column and input the correct rating also

In [12]:
df.iloc[5813, df.columns.get_loc('duration')] = "66 min"
df.iloc[5813, df.columns.get_loc('rating')] = "TV-MA"
df.iloc[5794, df.columns.get_loc('duration')] = "84 min"
df.iloc[5794, df.columns.get_loc('rating')] = "TV-MA"
df.iloc[5541, df.columns.get_loc('duration')] = "74 min"
df.iloc[5541, df.columns.get_loc('rating')] = "TV-MA"

In [13]:
df.iloc[5813]
df.iloc[5794]
df.iloc[5541]

In [14]:
print(df['duration'].isnull().sum())



We have dealt with mising values in the duration column, now let's turn our attention back to ratings column, rather than searching the titles of the missing values, instead I am going to replace the misisng values with the most common; TV-MA

In [15]:
df["rating"].replace(np.nan, "TV-MA", inplace=True)

In [16]:
df['rating'].isnull().sum()

Great, another column down. Now let's take a look at Date_added.
The data in this column shows the date that the title was added to Netflix, this data may be hard to find, instead of wasting time, I'm going to remove these rows from the analysis.

In [17]:
df.dropna(subset=["date_added"], axis=0, inplace=True)
# reset index, because we droped some rows
df.reset_index(drop=True, inplace=True)

In [18]:
df['date_added'].isnull().sum()

Now we see that we also have no null rows in the Date Added column, great. For the columns Director, Cast and Country that still contain missing data, I will cgoose to ignore these. Now that I am happy with the state of the data I can begin my analsysis.

The aim of this project is to explore the data that is available on Netflix, from the column names we know that Netlix offers TV-Shows and Movies. But in what proportion?

In [19]:
pie_data = df.groupby(['type'])['show_id'].count().reset_index()
pie_data

In [20]:
fig = px.pie(pie_data, values='show_id', names='type', title='Proportion of Movies vs TV Shows on Netflix') 
fig.show()

This pie chart shows us that the majority of the Netflix catalogue consists of movies, about a 70:30 split between movies and TV Shows. Given that the majority of the titles on Netflix, let's take a closer look at them.

In [21]:
hist_data = df.groupby(['release_year','type'])['show_id'].count().reset_index()
hist_data.head()

In [22]:
fig = px.bar(hist_data, x="release_year", y='show_id', color="type")
fig.show()

This is a bar chart to show the the amount of movies and TV-shows on Netflix by release year. On the x-axis we have release year and on the y-axis the sum of titles released, with Movies indicated by the red bar and TV-shows indicated by thr blue .
From the histogram, we can see that the Netflix catalogue has a range of movies from 1942, to 2021 with the majority of the movies being released in the year 2021 

No we have an idea of the quantity of movies and there release dates I'd like to see if there is any information that can be 
gathered from the country data.
I begin by creating a new dataframe of just the countries 

In [23]:
countries = df['country'].value_counts()

countries.head(10)

Now I want to create a list of 

In [24]:
country_list = ['United States', 'India', 'United Kingdom', 'Japan', 'South Korea', 'Canada',
              'Spain', 'France', 'Mexico', 'Egypt']

I want to drop the countries that aren't in the top 10 country list and rename the show ID column to count or sum.

In [25]:
country_df = df[df['country'].isin(country_list)].reset_index()
country_df =country_df.groupby(['country','type'])['show_id'].count().reset_index()
country_df.head()

In [26]:
fig = px.bar(country_df, x="country", y="show_id",
             color='type', barmode='group')
fig.show()

This graph shows the top 10 content makers for Netflix and that the United States produces the most content on Netflix 
For TV shows this is followed secondly by India, then the UK
In 10th place for movies we have Egypt

In [27]:
df.dtypes

In [28]:
df['year_added'] = str(df['date_added'])

In [29]:
df.dtypes

In [30]:
df['date_added'] = pd.to_datetime(df['date_added'])



In [31]:
df.dtypes

In [32]:
import datetime

In [33]:
df['time'] = pd.DatetimeIndex(df['date_added']).year


In [34]:
df['time'].head(10)

In [35]:
df['long'] = df['time']-df['release_year']

In [36]:
df['long']

In [37]:
df.drop(df[df.long < 0].index, inplace=True)

In [38]:
df['long'].describe()

In [39]:
time_to_screen = df.groupby(['long'])['show_id'].count().reset_index()

fig = px.bar(time_to_screen, x='long', y = 'show_id')
fig.show()

Here we can see that the majority of Netflix shows and movies are relased and are on Netflix within one year, this is likely due to the productions released by Netflix Studios 

Ensuring a regurlar catalogue of new shows entices the viewers to keep coming back to Netflix and keeps retention 